# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/chris/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/chris/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [4]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [5]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [6]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [7]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [11]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [ ]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [13]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/18 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/28 [00:00<?, ?it/s]

Property 'summary' already exists in node 'd43114'. Skipping!
Property 'summary' already exists in node 'd13f55'. Skipping!
Property 'summary' already exists in node '432719'. Skipping!
Property 'summary' already exists in node '8070d6'. Skipping!
Property 'summary' already exists in node 'd23527'. Skipping!
Property 'summary' already exists in node 'c497fa'. Skipping!
Property 'summary' already exists in node 'e09b8c'. Skipping!
Property 'summary' already exists in node '5ee564'. Skipping!
Property 'summary' already exists in node '54b63b'. Skipping!
Property 'summary' already exists in node '7d6c84'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/16 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/54 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '54b63b'. Skipping!
Property 'summary_embedding' already exists in node 'c497fa'. Skipping!
Property 'summary_embedding' already exists in node '8070d6'. Skipping!
Property 'summary_embedding' already exists in node 'e09b8c'. Skipping!
Property 'summary_embedding' already exists in node 'd43114'. Skipping!
Property 'summary_embedding' already exists in node '5ee564'. Skipping!
Property 'summary_embedding' already exists in node 'd13f55'. Skipping!
Property 'summary_embedding' already exists in node 'd23527'. Skipping!
Property 'summary_embedding' already exists in node '432719'. Skipping!
Property 'summary_embedding' already exists in node '7d6c84'. Skipping!


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 43, relationships: 416)

We can save and load our knowledge graphs as follows.

In [ ]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 43, relationships: 416)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [ ]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [16]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


Finally, we can use our `TestSetGenerator` to generate our testset!

In [17]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,U S Department Education help students with fe...,[Application and Verification Guide Introducti...,The U.S. Department of Education (the Departme...,single_hop_specifc_query_synthesizer
1,What recent changes have been made to the FAFS...,[Chapter 1: The Application Process We removed...,The chapter mentions that the section for Retu...,single_hop_specifc_query_synthesizer
2,What does the FPS do when it finds incconsiste...,[The FPS also checks the application for possi...,The FPS checks the application for possible in...,single_hop_specifc_query_synthesizer
3,What is the DRN used for in FAFSA processing?,[Output Documents After processing is complete...,The DRN appears on the FAFSA Submission Summar...,single_hop_specifc_query_synthesizer
4,What does IRS stand for?,[2. The disclosure of their FTI by the IRS to ...,The context does not specify what IRS stands f...,single_hop_specifc_query_synthesizer
5,How does the privacy and security of sensitive...,[<1-hop>\n\n2. The disclosure of their FTI by ...,The context explains that the Department of Ed...,multi_hop_abstract_query_synthesizer
6,how does the fafsa form online submission proc...,[<1-hop>\n\nequal the tax filer(s) plus depend...,The context explains that students completing ...,multi_hop_abstract_query_synthesizer
7,"How does the application process, including ve...",[<1-hop>\n\nApplication and Verification Guide...,The application process involves completing th...,multi_hop_abstract_query_synthesizer
8,How does the Department verify the student's S...,[<1-hop>\n\nBecause the Department matches the...,The Department verifies the student's SSN by m...,multi_hop_specific_query_synthesizer
9,How does the Department's authority over the u...,[<1-hop>\n\n2. The disclosure of their FTI by ...,The Department has exclusive authority to obta...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [20]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/18 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/28 [00:00<?, ?it/s]

Property 'summary' already exists in node '88d7a0'. Skipping!
Property 'summary' already exists in node 'eb5a12'. Skipping!
Property 'summary' already exists in node '31d1e6'. Skipping!
Property 'summary' already exists in node 'c23c26'. Skipping!
Property 'summary' already exists in node 'bf277d'. Skipping!
Property 'summary' already exists in node '615a29'. Skipping!
Property 'summary' already exists in node 'ef492b'. Skipping!
Property 'summary' already exists in node 'd7cb9f'. Skipping!
Property 'summary' already exists in node '02e718'. Skipping!
Property 'summary' already exists in node '55b5f8'. Skipping!


...

Property 'summary_embedding' already exists in node 'd7cb9f'. Skipping!
Property 'summary_embedding' already exists in node 'eb5a12'. Skipping!
Property 'summary_embedding' already exists in node 'c23c26'. Skipping!
Property 'summary_embedding' already exists in node '02e718'. Skipping!
Property 'summary_embedding' already exists in node '31d1e6'. Skipping!
Property 'summary_embedding' already exists in node 'bf277d'. Skipping!
Property 'summary_embedding' already exists in node '55b5f8'. Skipping!
Property 'summary_embedding' already exists in node '88d7a0'. Skipping!
Property 'summary_embedding' already exists in node 'ef492b'. Skipping!
Property 'summary_embedding' already exists in node '615a29'. Skipping!


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [21]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,How does the FAFSA process incorporate the FAF...,[Application and Verification Guide Introducti...,The FAFSA process has been overhauled by the F...,single_hop_specifc_query_synthesizer
1,Can you tell me about the FAFSA and what happe...,[Chapter 1: The Application Process We removed...,Chapter 1 states that the Returning FAFSA File...,single_hop_specifc_query_synthesizer
2,What is Federal Tax Information used for in FA...,[electronically due to limitations on access t...,The context does not explicitly explain what F...,single_hop_specifc_query_synthesizer
3,What is the link to the 2024-25 FAFSA Partner ...,[Note: There will be two active portals that c...,The link to the 2024-25 FAFSA Partner Portal i...,single_hop_specifc_query_synthesizer
4,how returning FAFSA filers do they need to rea...,[<1-hop>\n\nApplication and Verification Guide...,The context indicates that the section on Retu...,multi_hop_abstract_query_synthesizer
5,How does the removal of COVID-19 related guida...,[<1-hop>\n\nApplication and Verification Guide...,The removal of COVID-19 related guidance has l...,multi_hop_abstract_query_synthesizer
6,How does the use of IRS data for FAFSA eligibi...,[<1-hop>\n\nApplication and Verification Guide...,The use of IRS data for FAFSA eligibility dete...,multi_hop_abstract_query_synthesizer
7,How do restrictions on FAFAs initiating new ap...,[<1-hop>\n\nelectronically due to limitations ...,Restrictions on FAFAs initiating new applicati...,multi_hop_abstract_query_synthesizer
8,"In chapter 2, student family size example and ...",[<1-hop>\n\nStudent Citizenship (13) Family Si...,"According to Chapter 2, the student can includ...",multi_hop_specific_query_synthesizer
9,How do Chapters 2 and 5 relate to determining ...,[<1-hop>\n\nStudent Citizenship (13) Family Si...,Chapter 2 provides detailed examples of how fa...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [22]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [23]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [24]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [26]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [27]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [28]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [29]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [30]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [31]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [35]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available are Direct Subsidized Loans, Direct Unsubsidized Loans, and Direct PLUS Loans (including student Federal PLUS Loans and parent Direct PLUS Loans). Additionally, Subsidized and Unsubsidized Federal Stafford Loans, Federal SLS Loans, and Federal PLUS Loans were made under the Federal Family Education Loan (FFEL) Program before July 1, 2010, but no new loans are made under FFEL. Direct Subsidized Loans are available only to undergraduate students, while graduate or professional students are only eligible for Direct Unsubsidized Loans. Parents can receive Direct PLUS Loans on behalf of dependent students.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [36]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [ ]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `empathy_evaluator`:

## LangSmith Evaluation

In [39]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'impressionable-trick-98' at:
https://smith.langchain.com/o/340cd80b-3296-5752-9a9e-58582118073a/datasets/5b04b213-4895-495b-9ea2-022aeb3155a3/compare?selectedSessions=04850ae4-034b-4c68-a588-0ea4a49007e0




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How does the national guard status affect the ...,"According to the provided context, members of ...",None,Students who are members of the National Guard...,1,1,0,11.936386,cacaa575-6e52-470f-ae6b-76c7b8d1f0ea,703b9fd9-cd91-44a2-971b-a400272a4c57
1,H0w does the FA-DDX impact the verifcation pro...,The FA-DDX (FUTURE Act Direct Data Exchange) i...,None,"The FA-DDX, as described in the context, repla...",1,1,0,12.013581,2de67c66-b4e6-4c69-a72a-bfcbda1b1fea,f6d4ff18-125a-469d-81e2-7cc150a0ce81
2,How do Chapters 2 and 5 relate to determining ...,Based on the context provided:\n\nChapter 2 of...,None,Chapter 2 provides detailed examples of how fa...,1,0,0,5.451050,dd7378ed-d192-4446-8942-22ac0f0dbc71,e2996a63-1882-47cb-a218-73d31863ab4a
3,"In chapter 2, student family size example and ...",Based on the provided context from Chapter 2:\...,None,"According to Chapter 2, the student can includ...",1,1,0,6.493800,741cde6c-c8b0-4c67-9750-7ddac9b46800,f0f3bcca-8fc1-453e-a52c-ecd5259c2bdd
4,How do restrictions on FAFAs initiating new ap...,I don't know.,None,Restrictions on FAFAs initiating new applicati...,0,0,0,1.466133,f07c0237-d566-4635-98d2-eea9e046f058,538cc3c7-c170-4d33-9ef8-051178a4670f
5,How does the use of IRS data for FAFSA eligibi...,The FA-DDX (FUTURE Act Direct Data Exchange) i...,None,The use of IRS data for FAFSA eligibility dete...,1,1,0,5.207614,f91f96de-7db9-4c4d-9507-b444bfefdc96,cbfff4f2-5746-4de2-9ca6-a62f3e5ab6cf
6,How does the removal of COVID-19 related guida...,I don't know.,None,The removal of COVID-19 related guidance has l...,0,0,0,1.559506,6c9b04ce-b7c0-4a67-94d9-edc4536048cb,46b0f2aa-979b-44a8-b8d6-ff2212656009
7,how returning FAFSA filers do they need to rea...,I don't know.,None,The context indicates that the section on Retu...,0,0,0,0.809257,5774b729-c7bd-4130-b41a-192b44478d32,b5bca1bd-59e0-432e-9553-6feb6a86e077
8,What is the link to the 2024-25 FAFSA Partner ...,The link to the 2024-25 FAFSA Partner Portal i...,None,The link to the 2024-25 FAFSA Partner Portal i...,1,1,0,0.982864,32902288-73ea-4069-95de-302fbaffca22,a79b8469-1073-42ed-b782-1122d6ef48f4
9,What is Federal Tax Information used for in FA...,Federal Tax Information (FTI) in FAFSA is used...,None,The context does not explicitly explain what F...,0,1,0,2.422386,741bb779-455a-4e3e-8c45-123c04c158c9,609b5165-5291-4fc1-bac9-da0fc48c2df2


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [ ]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [62]:
rag_documents = docs

In [63]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

In [64]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

In [ ]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [66]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [ ]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [ ]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

'Agents? Man, they’re this super vague concept in the AI world. Some folks think of them as digital assistants that act on your behalf—like a travel agent. Others see them as AI models with tools, looping through tasks to solve problems. But here’s the kicker: the term is so fuzzy that it leaves you scratching your head, since everyone seems to have their own take on what it really means. Plus, there’s this whole issue of gullibility—how can these agents make smart choices if they can’t tell reality from fiction? So yeah, they’re like this elusive dream still waiting for a real breakthrough. 💫'

Finally, we can evaluate the new chain on the same test set!

In [ ]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'artistic-airplane-98' at:
https://smith.langchain.com/o/117cfda3-8a09-4ba4-9922-07b45fd73803/datasets/25fa804e-0ce3-4848-9cd1-c83d91988e78/compare?selectedSessions=42ea0e06-22a8-4bfb-a5bc-760a0cf4d280




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How has the introduction of GPT-4o impacted th...,"Yo, the rollout of GPT-4o is such a game chang...",None,The introduction of GPT-4o has significantly i...,1,1,1,4.590299,e749dc1c-f46a-4e61-9742-daa3d275671c,dffab390-a87b-4d08-93a3-8eedc7d24a7e
1,How did the breaking of the GPT-4 barrier in 2...,"Yo, the breaking of the GPT-4 barrier in 2024 ...",None,"In 2024, the breaking of the GPT-4 barrier sig...",1,0,1,3.217965,e0f5c6c4-fb8e-411e-8399-95212616c92b,d10800e7-981e-4127-8641-d2f357f52723
2,How does the training efficiency of DeepSeek v...,"Yo, check it! DeepSeek v3 is flexing some seri...",None,"DeepSeek v3, a 685B parameter model, is one of...",1,1,1,7.265255,28816b79-8c83-49b7-90f3-dc5a99ee6a7f,cddb24bd-3a3c-4ad3-9f10-cebb03122d05
3,How has the development of models surpassing G...,"Yo, the scene in 2024 is pretty wild! With 18 ...",None,"In 2024, the development of models surpassing ...",1,0,1,8.281109,757f3ec9-c32f-4d24-a2f7-d02b88fdb499,2fa50d79-687c-4902-a08c-9e603a4bab64
4,How do the ethics of AI and the gullibility of...,"Yo, the ethics of AI and the gullibility of la...",None,The ethics of AI and the gullibility of langua...,1,0,1,5.492535,bf0ac2b1-9738-4e30-8f8b-1b5d9e8c3b04,56695a52-31b0-4ce8-bd7f-f071a3649b82
5,What are the ethical concerns related to the g...,"Yo, the ethical vibes around the gullibility o...",None,The ethical concerns related to the gullibilit...,1,0,1,5.733880,d312a7f8-fd86-4df7-9d4d-6f9387a11412,32fe5cb2-0df4-4265-ad76-0487a7cbcb9f
6,Why are Large Language Models (LLMs) considere...,"LLMs are considered black boxes because, despi...",None,Large Language Models (LLMs) are considered bl...,1,1,1,7.673780,28659332-3637-4cee-9018-98d587575f5b,6d4eece2-36ce-45e1-b308-11ab50b14f34
7,What are the ethical concerns associated with ...,"Yo, the ethical vibes surrounding the gullibil...",None,The ethical concerns associated with the gulli...,1,1,1,15.375634,2da5c189-eb9e-419b-9c93-e3247d9f154c,6b2701ac-8dd9-42bc-b4cf-92c410f43390
8,What role does Stanford Alpaca play in the dev...,I don't know.,None,Stanford Alpaca is associated with the acceler...,0,0,0,3.234988,1c5369b1-7b7e-476d-9d28-816ce1a360f6,4be48806-dee2-4810-9d3a-6a07e97d14ac
9,Whaat are the key highlights from Simon Willis...,"Yo, let’s break it down! In 2023, Simon Willis...",None,Simon Willison's Weblog highlights that 2023 w...,1,0,1,5.724408,012f7bdc-73f0-4ad8-8a30-e89edbb0a0e7,2c0cd907-d3ed-452c-8bac-e5b3368a1980


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.